In [1]:
from langchain_groq import ChatGroq
import os,getpass, time
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from langchain_pinecone import PineconeVectorStore



/usr/local/python/3.10.13/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")


In [3]:
os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key:  ")


In [7]:
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0.2,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [14]:
prompt_template = """Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.

{context}

Question: {question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [8]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [9]:
index_name = 'pinecone-rag'

In [10]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 16}},
 'total_vector_count': 16}

In [11]:
import pickle

# Load embeddings
with open("huggingface_embeddings.pkl", "rb") as f:
    huggingface_embeddings = pickle.load(f)


In [12]:
vectordb = PineconeVectorStore(index=index, embedding=huggingface_embeddings)


In [15]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [16]:
query = "what is the formula for the fields equation?"

In [17]:
result = retrievalQA.invoke({"query": query})
print(result['result'])

The formula for Einstein's field equations is expressed as: Gμnu + Λgμnu = (8πG / c^4) Tμnu. Here, Gμnu represents the Einstein tensor that encodes the curvature of spacetime due to gravity, Λ is the cosmological constant that represents the energy density of empty space, gμnu is the metric tensor describing the shape of spacetime, and Tμnu signifies the energy-momentum tensor that gives the distribution of matter and energy in spacetime.


### testing the RAG application:

In [18]:
green = "\033[0;32m"


In [20]:
print(f"{green}------------------------------------------------------------------------")
print(f"Testing the RAG application with Einstein's Field Equations as the KB")
print('---------------------------------------------------------------------------------------')
for i in range(1):
  user_input = input("type here: ")
  result = retrievalQA.invoke({"query": user_input})
  print("Query:  ", user_input)
  print("Bwhiz Bot: ",result['result'])


------------------------------------------------------------------------
Testing the RAG application with Einstein's Field Equations as the KB
---------------------------------------------------------------------------------------
Query:   what is the formula for the fields equation?
Bwhiz Bot:  The formula for Einstein's field equations is expressed as: Gμnu + Λgμnu = (8πG / c^4) Tμnu. Here, Gμnu represents the Einstein tensor that encodes the curvature of spacetime due to gravity, Λ is the cosmological constant that represents the energy density of empty space, gμnu is the metric tensor describing the shape of spacetime, and Tμnu denotes the energy-momentum tensor that describes the density and flux of energy and momentum.
